In [1]:
import h5py
import numpy as np

In [2]:
def merge_dataset_chunked(f_in, f_out, chunk_size=20):
    # Copy 'train' datasets if not exist in output, in chunks
    for data_name in f_in['train']:
        if data_name not in f_out:
            ds_in = f_in['train'][data_name]
            max_shape = ds_in.shape
            chunk_shape = (min(chunk_size, max_shape[0]),) + max_shape[1:]
            ds_out = f_out.create_dataset(data_name, shape=max_shape, dtype=ds_in.dtype, chunks=chunk_shape)
            for i in range(0, ds_in.shape[0], chunk_size):
                ds_out[i:i + chunk_size] = ds_in[i:i + chunk_size]
    
    for group_name in ['test', 'valid']:
        if group_name in f_in:
            for data_name in f_in[group_name]:
                ds_in = f_in[group_name][data_name]
                if data_name in f_out:
                    ds_out = f_out[data_name]
                    new_shape = (ds_out.shape[0] + ds_in.shape[0],) + ds_out.shape[1:]
                    temp_name = data_name + "_temp"
                    temp_ds = f_out.create_dataset(temp_name, new_shape, dtype=ds_in.dtype, chunks=ds_out.chunks)
                    
                    # Copy data from the old dataset in chunks
                    for i in range(0, ds_out.shape[0], chunk_size):
                        end_index = min(i + chunk_size, ds_out.shape[0])
                        temp_ds[i:end_index] = ds_out[i:end_index]
                    
                    del f_out[data_name]  # Remove old dataset
                    
                    # Append new data in chunks
                    for i in range(0, ds_in.shape[0], chunk_size):
                        end_index = ds_out.shape[0] + i + chunk_size
                        temp_ds[ds_out.shape[0] + i:end_index] = ds_in[i:i + chunk_size]
                    
                    f_out.move(temp_name, data_name)
                    del temp_ds
                else:
                    max_shape = ds_in.shape
                    chunk_shape = (min(chunk_size, max_shape[0]),) + max_shape[1:]
                    f_out.create_dataset(data_name, shape=max_shape, dtype=ds_in.dtype, chunks=chunk_shape, data=ds_in)


In [3]:
def copy_datasets(source_file_path, target_file_path):
    with h5py.File(source_file_path, 'r') as source_file, h5py.File(target_file_path, 'w') as target_file:
        def copy_dataset(name, obj):
            if isinstance(obj, h5py.Dataset):
                target_file.create_dataset(name, data=obj[()])
            elif isinstance(obj, h5py.Group):
                target_file.create_group(name)
        
        source_file.visititems(copy_dataset)

In [4]:
input_file = "../../data/ns_contextual/ns_random_forces.h5"
output_file = "../../data/ns_contextual/ns_random_forces_v1.h5"

with h5py.File(input_file, 'r') as f_in:
    with h5py.File(output_file, 'a') as f_out:
        merge_dataset_chunked(f_in, f_out)

        # Read the data into memory
        a_data = f_out['a'][:]
        u_data = f_out['u'][:]

        # Process the data
        expanded_a_data = np.expand_dims(a_data, axis=-1)
        concatenated_data = np.concatenate((expanded_a_data, u_data), axis=-1)

        # Replace the 'u' dataset
        del f_out['u']  # Delete the existing 'u' dataset
        f_out.create_dataset('u', data=concatenated_data)

        # Optionally, delete the 'a' dataset if no longer needed
        del f_out['a']

: 

In [ ]:
source_h5_path = "../../data/ns_contextual/ns_random_forces_v1.h5"
target_h5_path = "../../data/ns_contextual/ns_random_forces_v0.h5"
copy_datasets(source_h5_path, target_h5_path)

In [ ]:
!rm "../data/ns_contextual/ns_random_forces_v1.h5"